In [1]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from spacy.training import offsets_to_biluo_tags

# Dados de treinamento
TRAIN_DATA = [
    ("Quais foram os 5 maiores fornecedores de janeiro a março de 2024?", {"entities": [(15,24, "MAX_MIN"), (25,37, "TABELA"), (41,64, "INTERVALO")]}),
    ("Qual o total e percentual de pagamentos manuais ocorridos em janeiro de 2024?", {"entities": [(10,15, "TABELA"), (18,28, "TABELA"), (32,51, "CONDICAO"), (63,77, "INTERVALO")]}),
    ("Quanto tenho a pagar para o fornecedor Google?", {"entities": [(13,21, "CONDICAO"), (28,38, "TABELA"), (39,45, "FORNECEDOR")]}),
    ("Quanto tenho a pagar para os fornecedores X, Y, Z?", {"entities": [(13,21, "CONDICAO"), (29,41, "TABELA"), (39,40, "FORNECEDOR"), (42,43, "FORNECEDOR"), (45,46, "FORNECEDOR")]}),
    ("Quanto tenho a pagar para fornecedores do grupo 4000 (Jurídico)?", {"entities": [(13,21, "CONDICAO"), (26,38, "TABELA"), (48,52, "ID")]}),
    ("Quanto paguei ontem para a empresa 1000?", {"entities": [(7,13, "CONDICAO"), (14,19, "INTERVALO"), (35,39, "ID")]}),
    ("Qual o total que tenho a pagar para a empresa 1100 hoje?", {"entities": [(23,29, "CONDICAO"), (46,50, "ID"), (51,55, "INTERVALO")]}),
    ("Pagamentos manuais ou pagamento manual (Z, E, B).", {"entities": [(0,10, "TABELA"), (11,18, "CONDICAO"), (40,41, "FORNECEDOR"), (43,44, "FORNECEDOR"), (46,47, "FORNECEDOR")]}),
    ("Pagamentos (total, listagem de, etc.) efetuados após o vencimento, considerando-se calendário FEBRABAN, por empresa e intervalo de tempo.", {"entities": [(0,10, "TABELA"), (48,65, "CONDICAO")]}),
    ("Percentual de pagamentos efetuados após o vencimento, considerando o total de vencimentos.", {"entities": [(14,24, "TABELA"), (35,52, "CONDICAO")]}),
    ("Documentos vencidos, por empresa, por intervalo de tempo, por fornecedor, etc., considerando-se o calendário FEBRABAN.", {"entities": [(0,10, "TABELA"), (11,19, "CONDICAO")]}),
    ("Relação de documentos (ou total) a vencer (considerando calendário FEBRABAN) em dado período de tempo.", {"entities": [ (11,21, "TABELA"), (33,41, "CONDICAO")]}),
    ("Qual o total de pagamentos efetuados após o vencimento pela empresa X no período de janeiro a março de 2024?", {"entities": [(16,26, "TABELA"), (38,54, "CONDICAO"), (68,69, "FORNECEDOR"), (84,107, "INTERVALO")]}),
    ("Listagem de pagamentos efetuados após o vencimento para o fornecedor Y considerando calendário FEBRABAN.", {"entities": [ (12,22, "TABELA"), (33,50, "CONDICAO"), (69,70, "FORNECEDOR")]}),
    ("Percentual de documentos vencidos para o grupo 4000 no primeiro trimestre de 2024.", {"entities": [(14,24, "TABELA"), (25,33, "CONDICAO"), (47,51, "ID"), (55,73, "INTERVALO")]}),
    ("Relação de documentos a vencer para o fornecedor Google no mês de fevereiro de 2024.", {"entities": [(11,21, "TABELA"), (22,30, "CONDICAO"), (49,55, "FORNECEDOR"), (66,83, "INTERVALO")]}),
    ("Total de pagamentos vencidos por empresa 1000 no ano de 2023.", {"entities": [(9,19, "TABELA"), (20,28, "CONDICAO"), (41,45, "ID"), (49,62, "INTERVALO")]}),
    ("Percentual de pagamentos manuais feitos para os fornecedores X, Y e Z em janeiro de 2024.", {"entities": [(14,24, "TABELA"), (25,32, "CONDICAO"), (43,55, "TABELA"), (61,62, "FORNECEDOR"), (64,65, "FORNECEDOR"), (68,69, "FORNECEDOR"), (73,88, "INTERVALO")]}),
    ("Quais são os documentos a vencer para o grupo Jurídico na próxima semana?", {"entities": [(13,23, "TABELA"), (24,32, "CONDICAO"), (46,54, "ID"), (58,72, "INTERVALO")]}),
    ("Qual o total de pagamentos efetuados em janeiro de 2024?", {"entities": [(16,27, "TABELA"), (40,55, "INTERVALO")]}),
    ("Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?", {"entities": [(10,20, "TABELA"), (24,34, "TABELA"), (35,42, "CONDICAO"), (49,65, "INTERVALO")]}),
    ("Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?", {"entities": [(21,31, "TABELA"), (32,39, "CONDICAO"), (50,67, "INTERVALO")]}),
    ("Qual foi o valor total pago para o fornecedor Microsoft no mês de março de 2024?", {"entities": [(35,45, "TABELA"),(46,55, "FORNECEDOR"), (66,79, "INTERVALO")]}),
    ("Quanto tenho a pagar para fornecedores do grupo 3000 (TI)?", {"entities": [(13,21, "CONDICAO"), (26,38, "TABELA"), (48,52, "ID")]}),
    ("Quanto paguei anteontem para a empresa 2000?", {"entities": [(7,13, "CONDICAO"), (14,23, "INTERVALO"), (39,43, "ID")]}),
    ("Qual o total que tenho a pagar para a empresa 2100 amanhã?", {"entities": [(23,30, "CONDICAO"), (46,50, "ID"), (51,57, "INTERVALO")]}),
    ("Pagamentos manuais efetuados para (A, B, C) no primeiro trimestre de 2024.", {"entities": [(0,10, "TABELA"), (11,18, "CONDICAO"), (35,36, "FORNECEDOR"), (38,39, "FORNECEDOR"), (41,42, "FORNECEDOR"), (47,73, "INTERVALO")]}),
    ("Quais foram os 10 maiores fornecedores de fevereiro a abril de 2024?", {"entities": [(15,25, "MAX_MIN"), (26,39, "TABELA"), (42,67, "INTERVALO")]}),
    ("Quanto foi pago em junho de 2024 para fornecedores do grupo 5000 (Financeiro)?", {"entities": [(38,50, "TABELA"), (19,32, "INTERVALO"), (60,64, "ID")]}),
    ("Qual o valor total de pagamentos vencidos para a empresa 3100 em maio de 2024?", {"entities": [(22,33, "TABELA"), (33,41, "CONDICAO"), (57,61, "ID"), (65,77, "INTERVALO")]}),
    ("Qual é a lista de fornecedores do grupo 6000 (Marketing) no mês de julho de 2024?", {"entities": [(18,30, "TABELA"), (40,44, "ID"), (67,80, "INTERVALO")]}),
    ("Quantos documentos venceram em agosto de 2024 para o fornecedor Amazon?", {"entities": [(8,18, "TABELA"), (19,27, "CONDICAO"), (31,45, "INTERVALO"), (64,70, "FORNECEDOR")]}),
    ("Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.", {"entities": [(11,21, "TABELA"), (22,30, "CONDICAO"), (34,50, "INTERVALO")]}),
    ("Qual é a relação de documentos a vencer para a empresa 9000 no próximo semestre?", {"entities": [(20,30, "TABELA"), (31,39, "CONDICAO"), (55,60, "ID"),(63,79, "INTERVALO")]}),
    ("Quantos pagamentos manuais foram feitos para o fornecedor Oracle no primeiro trimestre de 2023?", {"entities": [(8,26, "CONDICAO"), (47,57, "TABELA"), (58,64, "FORNECEDOR"),(68,94, "INTERVALO")]}),
    ("Qual o total de documentos a vencer no mês de fevereiro de 2023 para o grupo 1200 (Comercial)?", {"entities": [(16,26, "TABELA"), (46,64, "INTERVALO"), (77,81, "ID")]}),
    ("Qual o percentual de pagamentos efetuados após o vencimento para a empresa 8000 no quarto trimestre de 2024?", {"entities": [(21,31, "TABELA"), (42,59, "CONDICAO"), (75,79, 'ID'), (83,107, "INTERVALO")]}),
    ("Quanto foi pago para fornecedores do grupo 1100 (Administrativo) no último semestre?", {"entities": [(21,33, "TABELA"), (43,47, "ID"), (75,79, 'ID'), (68,83, "INTERVALO")]}),
    ("Relação de documentos vencidos para a empresa 7000 considerando o calendário FEBRABAN.", {"entities": [(11,21, "TABELA"), (22,30, "CONDICAO"), (46,50, 'ID')]}),
    ("Quanto tenho a pagar para fornecedores do grupo 1000 (TI) no próximo trimestre?", {"entities": [(13,21, "CONDICAO"), (26,38, "TABELA"), (48,52, 'ID'), (61,78, 'INTERVALO')]}),
    ("Quais são os fornecedores do grupo 9000 (Logística) que têm documentos a vencer no próximo mês?", {"entities": [(13,25, "TABELA"), (35,39, "ID"), (60,70, "TABELA"), (71,79, 'CONDICAO'), (83,94, 'INTERVALO')]}),
    ("Qual o valor total de documentos vencidos para o fornecedor IBM no ano de 2022?", {"entities": [(22,32, "TABELA"), (33,41, "CONDICAO"), (60,63, "FONECEDOR"), (67,78, 'INTERVALO')]}),
    ("Quantos pagamentos manuais foram efetuados para a empresa 6000 no mês passado?", {"entities": [(8,18, "TABELA"), (19,26, "CONDICAO"), (58,63, "ID"), (66,77, 'INTERVALO')]}),
    ("Qual é o percentual de pagamentos efetuados manualmente para fornecedores do grupo 8000 (Operacional) em dezembro de 2024?", {"entities": [(23,33, "TABELA"), (44,55, "CONDICAO"), (83,87, "ID"), (105,121, 'INTERVALO')]}),
    ("Quanto foi pago para o fornecedor Facebook no ano de 2023?", {"entities": [(23,33, "TABELA"), (34,42, "FONECEDOR"), (46,57, "INTERVALO")]}),
    ("Qual o total de documentos a vencer em novembro de 2024 para a empresa 5000?", {"entities": [(16,26, "TABELA"), (27,36, "CONDICAO"), (39,55, "INTERVALO"), (71,75, 'ID')]}),
    ("Quais são os fornecedores que mais receberam pagamentos manuais em outubro de 2024?", {"entities": [(13,26, "TABELA"), (30,45, "MAX_MIN"), (45,55, "TABELA"), (56,63, 'CONDICAO'),(67,82, 'INTERVALO')]}),
    ("Percentual de documentos vencidos para o grupo 7000 (RH) no segundo trimestre de 2024.", {"entities": [(14,24, "TABELA"), (25,34, "CONDICAO"), (47,51, "ID"), (56,63, 'CONDICAO'),(57,85, 'INTERVALO')]}),
    ("Qual o total de pagamentos efetuados após o vencimento pela empresa 4000 no período de abril a junho de 2024?", {"entities": [(16,26, "TABELA"), (37,54, "CONDICAO"), (68,72, "ID"), (87,108, 'INTERVALO')]}),
    ("Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.", {"entities": [(11,21, "TABELA"), (22,30, "CONDICAO"), (34,50, 'INTERVALO')]}),
]

# Carrega um modelo base ou cria um modelo em branco
nlp = spacy.blank("pt")

# Adiciona o componente NER ao pipeline, se ainda não estiver presente
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")

# Adiciona os novos labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Função para verificar o alinhamento das entidades
def verify_alignment(nlp, text, entities):
    doc = nlp.make_doc(text)
    try:
        tags = offsets_to_biluo_tags(doc, entities)
        return True
    except Exception as e:
        return False

# Filtrar exemplos desalinhados
aligned_train_data = [(text, annotations) for text, annotations in TRAIN_DATA if verify_alignment(nlp, text, annotations["entities"])]

# Preparar os exemplos de treinamento
examples = []
for text, annotations in aligned_train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)

# Inicia o treinamento do modelo
optimizer = nlp.begin_training()

# Treinar o modelo
for epoch in range(20):  # Ajuste o número de épocas conforme necessário
    losses = {}
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(batch, sgd=optimizer, losses=losses)
    print(f"Epoch {epoch} - Losses: {losses}")

# Salvar o modelo treinado
nlp.to_disk("modelo_customizado")

# Carrega o modelo treinado
nlp = spacy.load("modelo_customizado")

# Testa o modelo em um novo texto
test_texts = [
    'Quais foram os fornecedores que mais venderam no último trimestre?',
    'Quantos pagamentos foram feitos manualmente em março deste ano?',
    'Qual a lista de fornecedores do grupo 2000?',
    'Como está o percentual de documentos vencidos este mês?',
    'Quem são os fornecedores com maior número de pagamentos em atraso?',
    'Quantos pagamentos foram efetuados após o vencimento no primeiro semestre de 2024?',
    'Qual é a relação de documentos a vencer em julho de 2024?'
]

for text in test_texts:
    doc = nlp(text)
    print(f"Text: {text}")
    for ent in doc.ents:
        print(f" - {ent.text}: {ent.label_}")

/opt/conda/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Qual o total e percentual de pagamentos manuais oc..." with entities "[(10, 15, 'TABELA'), (18, 28, 'TABELA'), (32, 51, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Quanto tenho a pagar para o fornecedor Google?" with entities "[(13, 21, 'CONDICAO'), (28, 38, 'TABELA'), (39, 45...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned

Epoch 0 - Losses: {'ner': 493.56845748377964}
Epoch 1 - Losses: {'ner': 198.49492981212097}
Epoch 2 - Losses: {'ner': 144.93132495808482}
Epoch 3 - Losses: {'ner': 129.47674446602923}
Epoch 4 - Losses: {'ner': 63.4072464447535}
Epoch 5 - Losses: {'ner': 48.10196258965395}
Epoch 6 - Losses: {'ner': 22.01660705337583}
Epoch 7 - Losses: {'ner': 14.80950692760056}
Epoch 8 - Losses: {'ner': 14.401789316343422}
Epoch 9 - Losses: {'ner': 8.38026284479437}
Epoch 10 - Losses: {'ner': 7.787344482709597}
Epoch 11 - Losses: {'ner': 5.609530577309061}
Epoch 12 - Losses: {'ner': 4.555462424767812}
Epoch 13 - Losses: {'ner': 6.509703685676698}
Epoch 14 - Losses: {'ner': 6.408724782524713}
Epoch 15 - Losses: {'ner': 4.16885074049357}
Epoch 16 - Losses: {'ner': 1.1469825492501267}
Epoch 17 - Losses: {'ner': 5.534367642426331}
Epoch 18 - Losses: {'ner': 1.5385797544582727}
Epoch 19 - Losses: {'ner': 2.6111416335257913}
Text: Quais foram os fornecedores que mais venderam no último trimestre?
 - fornecedo